In [1]:
# Import needed packages
import math
import sys
sys.path.append('../../darknight/')
from fcts import remove_space, r2pcorr, struc2mol
from usefct import difference, vector_magnitude, vector_angle, distribution
import numpy as np
import pandas as pd
import darkchem
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
get_ipython().run_line_magic('matplotlib', 'inline')
from sklearn.linear_model import LinearRegression
from rdkit import Chem
from rdkit.Chem.Draw import MolToImage
from rdkit.Chem import PandasTools,Draw
# plotting stuff
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import SVG, display
%config InlineBackend.figure_format = 'retina'
import statistics
from IPython.display import display_html

Using TensorFlow backend.


In [3]:
# Load model which generate the latent space 
model = darkchem.utils.load_model('../../../DarKnight_File/Final Trained DarkChem Network Weights/N7b_[M+H]/')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
# Load training data
data = pd.read_excel('ketone_alcohol/ketone_alcohol_train.xlsx')

# Remove the redundant space in the smiles string
data = remove_space(data)

# Transform reactants and products into dictionary
katrain = {'reactants': data['Reactants'], 'products': data['Products']}

# Calculate the vectors of reactants and products
rvec = [darkchem.utils.struct2vec(reactant) for reactant in katrain['reactants']]
pvec = [darkchem.utils.struct2vec(product) for product in katrain['products']]

# Transform the data type to np.array 
rvec = np.array(rvec).astype(int)
pvec = np.array(pvec).astype(int)

# Generate the latent space representation for reactants and products
r_latent = model.encoder.predict(rvec)
p_latent = model.encoder.predict(pvec)

# Transform the data type to DataFrame 
rvecdf = pd.DataFrame(r_latent)
pvecdf = pd.DataFrame(p_latent)

# Calculate the latent path vector for those 17 reactions
path = pvecdf - rvecdf

In [5]:
# Calculation of Pearson correlation coefficients between the different reaction vectors
pd.DataFrame(np.corrcoef(path).view(type=np.matrix))

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,1.000000,-0.168280,0.047439,0.123185,0.002316,0.089254,0.008819,0.034375,0.083315,-0.385076,...,0.086567,-0.359880,-0.206735,-0.408826,-0.204234,0.235952,-0.035966,-0.280958,0.101838,-0.342929
1,-0.168280,1.000000,-0.216220,0.088389,0.569001,0.080017,-0.038324,0.104468,0.095903,-0.231269,...,0.641948,0.501958,-0.015641,-0.122898,0.391183,0.244432,0.123211,0.302416,-0.343177,0.244501
2,0.047439,-0.216220,1.000000,-0.135308,0.199575,-0.040116,0.006691,-0.109342,0.054543,-0.052615,...,-0.120809,-0.254640,0.543786,-0.078843,0.077808,0.383479,-0.376950,-0.532842,0.294891,-0.442337
3,0.123185,0.088389,-0.135308,1.000000,0.077236,0.208273,0.049537,-0.109653,0.045653,0.017567,...,0.146458,-0.041303,-0.156376,0.100854,-0.018500,-0.064814,0.090268,0.083043,-0.097388,0.037603
4,0.002316,0.569001,0.199575,0.077236,1.000000,0.063210,-0.045365,0.152726,-0.085206,-0.246237,...,0.484556,-0.033388,0.281449,-0.157148,0.618594,0.583431,-0.161201,-0.333579,-0.039573,-0.255369
5,0.089254,0.080017,-0.040116,0.208273,0.063210,1.000000,0.140862,0.055085,0.095391,-0.005539,...,0.135784,-0.041540,0.195486,0.115012,0.005713,0.030093,0.047652,-0.064385,0.037444,-0.021699
6,0.008819,-0.038324,0.006691,0.049537,-0.045365,0.140862,1.000000,0.306158,0.133850,-0.000315,...,0.202763,-0.026438,0.111270,0.071626,0.055947,0.036861,0.014008,0.052088,0.042167,-0.124051
7,0.034375,0.104468,-0.109342,-0.109653,0.152726,0.055085,0.306158,1.000000,-0.031010,-0.204377,...,0.398802,0.197630,0.038861,-0.143271,0.387341,0.037502,0.077830,-0.210771,0.273333,-0.261856
8,0.083315,0.095903,0.054543,0.045653,-0.085206,0.095391,0.133850,-0.031010,1.000000,-0.494687,...,0.065196,0.103984,0.285173,-0.305979,0.084236,0.028675,0.018352,0.216733,-0.286320,0.125016
9,-0.385076,-0.231269,-0.052615,0.017567,-0.246237,-0.005539,-0.000315,-0.204377,-0.494687,1.000000,...,-0.141321,-0.169454,-0.001578,0.486921,-0.309082,-0.158320,-0.043741,0.033385,-0.040054,0.013228
